In [1]:
#This program we used to pull every two seasons in pulls, why two seasons? because internet sometimes used to
#be bad so we would not risk 5 pulses and let lags ruin 5 seasons pulling.
#we used beautiful soup here to obtain from the texts we got by the selenium [the hrefs of every game of every season]
#and ran on each, on function named: run_page, we took the specific information we wanted and have it all manipulated by the way
#we wanted it to be, for instance, taking year out of '2000/01' to 2000...
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [2]:
def create_BeautifulSoup_For_This_page(this_url):
    url=this_url
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

In [3]:
months_to_numerical_values_dict = {
    'jan':1,
     'feb':2,
     'mar':3,
     'apr':4,
     'may':5, 
     'jun':6,
     'jul':7,
     'aug':8,
     'sep':9,
     'oct':10,
     'nov':11,
     'dec':12
}

In [4]:
#getting info for capacity using the dictionary we created thanks to crawling wikipedia and getting capacity on each
#stadium in the premier league
stadiums_info_df = pd.read_csv('stadiums_info_filtered.csv')
stadiums_dict = dict(zip(stadiums_info_df.stadium, stadiums_info_df.capacity))

In [5]:
#getting info for club's city using the dictionary we created thanks to crawling wikipedia and getting capacity on each
#stadium in the premier league
clubs_to_cities_df = pd.read_csv('clubs_to_cities2.csv')
clubs_to_cities_dict = dict(zip(clubs_to_cities_df.club, clubs_to_cities_df.city))

In [6]:
#function to edit strings which has to get what beyond space included removed
def remove_what_beyond_space(new_string):
    index=new_string.find(' ')
    new_string=new_string[:index]
    return new_string

In [7]:
def is_it_derby(team_1, team_2):
    if(clubs_to_cities_dict[team_1] == clubs_to_cities_dict[team_2]):
        return 1
    return 0

In [8]:
def give_me_empty_data_frame():
    empty_lst = list()
    return pd.DataFrame({"team_1":empty_lst, "team_2":empty_lst, 
                 "team_1_rating":empty_lst,"team_2_rating":empty_lst,
                "capacity":empty_lst, "stadium":empty_lst,
                "month": empty_lst, "derby": empty_lst,
                "current_season": empty_lst, "year": empty_lst,
                "city": empty_lst, "temprature": empty_lst,
                "precipitation": empty_lst, "attendance":empty_lst})

In [9]:
def clear_string_for_month(month):
    month=month[6:10].strip()
    return month.lower()

In [10]:
def get_alias_for_this_team(current_team):
    return teams_alias[current_team]

In [24]:
#here we are going to find every element on the json file 
#here we created by soup and url dataframe FOR ONLY ONE PAGE, needs to be edited for all of the tables.
def run_page(soup, url, this_df):
    weather_dataframe_part_two='_weather_info.csv'
    div_soup = soup.find('div', {'class':'mcTabsContainer'})#detecting the json file
    json_file = json.loads(div_soup['data-fixture']) #loading the json_file from the web crawling
    try:
        atten=json_file['attendance']#attendance
    except: 
        atten=None
    team_1=json_file['teams'][0]['team']['name']#1 team
    team_2=json_file['teams'][1]['team']['name']#2 teams
    stadium=json_file['ground']['name']#stadium
    city=json_file['ground']['city']#city
    current_season=json_file['gameweek']['compSeason']['label']#1999/00
    short_name_team_1=json_file['teams'][0]['team']['club']['shortName']
    short_name_team_2=json_file['teams'][1]['team']['club']['shortName']
    month=json_file['kickoff']['label']
    derby=is_it_derby(team_1, team_2)
    
    #making month out of the whole thing
    month=clear_string_for_month(month)
    
    year = current_season[:4]
    weather_dataframe = pd.read_csv(city+weather_dataframe_part_two)
    
    temprature = weather_dataframe.at[months_to_numerical_values_dict[month]-1,'mean_temp']
    precipitation = weather_dataframe.at[months_to_numerical_values_dict[month]-1,'precipitation']

    web_ = give_me_link_to_get_current_table(soup)

    team_1_rank = team_rank_method(web_, short_name_team_1)
    team_2_rank = team_rank_method(web_, short_name_team_2)
    capacity = int(stadiums_dict[stadium].replace(',',''))
    this_df.loc[this_df.shape[0]] = [team_1, team_2, team_1_rank, team_2_rank, capacity, stadium,
                           month, derby, current_season, year, city, temprature, precipitation, atten]


In [12]:
#getting the team rank algorithm beneath, we just need href for the url we send so it will be done tomorrow.
def team_rank_method(url, team_name):
    soup_for_rating=create_BeautifulSoup_For_This_page(url)
    soup_for_rating.findAll('table')[1].findAll('tr')
    rank = give_me_this_team_current_rank(team_name, soup_for_rating)
    return rank
#what we need to do is get the link for each table in every game link we already have, find the href, then place this

In [13]:
#getting current team_rank on the table by tricky loop
#we ran on the match week table and found the current team's rating
def give_me_this_team_current_rank(team_name, soup):
    r=0
    for x in soup.findAll('table')[1].findAll('tr'):
        r+=1
        if(r==1):
            continue
        if(team_name in x.get_text()):
            return r-1

In [14]:
#trying to catch the href link run_page
#partial plan to get the current web of current table due to the current match week, so we could find out
#team rating eventually
def give_me_link_to_get_current_table(soup):
    #this_soup=create_BeautifulSoup_For_This_page(url)
    links_with_text = []
    for a in soup.find_all('a', href=True): 
        if '/matchweek' in a['href'] and 'table' in a['href']: 
            links_with_text.append(a['href'])
    site = 'https://www.premierleague.com'
    web_=site+links_with_text[0]
    return web_

In [25]:
#beta on one SEASON only!!!
######
#this_ = give_me_empty_data_frame()
url='https://www.premierleague.com/match/12485'
s=create_BeautifulSoup_For_This_page(url)
run_page(s, url, this_df_12)
#this_df_12 = give_me_empty_data_frame()

In [19]:
#Loop for one season test
this_df_11 = give_me_empty_data_frame()
line = 0
file_number=0
file_name = 'namesoftexts.txt'
with open(file_name) as f:
    for current_season in f:
        this_season_file_name = current_season.strip()
        line=0
        file_number+=1
        print("opened file number = {0}".format(file_number))
        with open(this_season_file_name) as s:
            for current_url in s:
                line+=1
                this_url = current_url.strip()
                soup = create_BeautifulSoup_For_This_page(this_url)
                run_page(soup, this_url, this_df_11)
                print(line, end =" ")
    print('Done')

opened file number = 1
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 2

In [20]:
#saving every pulse eventually
#this_df_11.to_csv('this_df_11.csv')

In [21]:
this_df_11

,team_1,team_2,team_1_rating,team_2_rating,capacity,stadium,month,derby,current_season,year,city,temprature,precipitation,attendance
0,Burnley,AFC Bournemouth,7.0,12.0,21944.0,Turf Moor,may,0.0,2017/18,2017,Burnley,10.0,79.3,20720.0
1,Crystal Palace,West Bromwich Albion,11.0,20.0,25486.0,Selhurst Park,may,0.0,2017/18,2017,London,14.0,52.1,25357.0
2,Huddersfield Town,Arsenal,16.0,6.0,24500.0,John Smith's Stadium,may,0.0,2017/18,2017,Huddersfield,10.0,26.0,24122.0
3,Liverpool,Brighton and Hove Albion,4.0,15.0,54074.0,Anfield,may,0.0,2017/18,2017,Liverpool,12.0,53.6,50752.0
4,Manchester United,Watford,2.0,14.0,75643.0,Old Trafford,may,0.0,2017/18,2017,Manchester,12.0,21.2,75049.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Everton,Stoke City,7.0,16.0,39414.0,Goodison Park,aug,0.0,2017/18,2017,Liverpool,16.0,66.6,39045.0
376,Southampton,Swansea City,11.0,12.0,32384.0,St. Mary's Stadium,aug,0.0,2017/18,2017,Southampton,17.0,10.9,31447.0
377,West Bromwich Albion,AFC Bournemouth,8.0,15.0,26445.0,The Hawthorns,aug,0.0,2017/18,2017,West Bromwich,16.0,27.0,25011.0
378,Brighton and Hove Albion,Manchester City,17.0,3.0,30666.0,Amex Stadium,aug,0.0,2017/18,2017,Falmer,17.0,24.0,30415.0
